<a href="https://colab.research.google.com/github/byiringiroscar/NLP_FELLOWSHIP/blob/main/Visit_Rwanda_Bot_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**BUILDING VISIT RWANDA CHATBOT**



In [ ]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.d0P8FRyix4/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.qDDGZwnt6t/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.EJCLIfyuar/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://deb.debian.org/debian bust

In [ ]:
import re
from urllib.parse import urljoin
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
import json
import time
import os
import requests
from urllib.parse import urlparse, unquote
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from selenium.webdriver.common.by import By
import pandas as pd
import multiprocessing as mp
from selenium.webdriver.support.ui import WebDriverWait
import time

chrome_options = Options()


service = Service(executable_path=r'/usr/bin/chromedriver')
chrome_options.add_argument('--disable-blink-features=AutomationControlled')
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')


driver = webdriver.Chrome(service=service, options=chrome_options)

In [ ]:
#Fetch the content from the front page
url_links = 'https://www.visitrwanda.com/'
content = requests.get(url_links).content

In [ ]:
#Initiating the BS4 parser
page = BeautifulSoup(content, 'html.parser')

In [ ]:
navigation = page.find("div", class_="header-menu")
link_nav = navigation.find_all("a")
nav_links = []
for link in link_nav:
  nav_links.append(link['href'])


In [ ]:
len(nav_links)

9

In [ ]:
def get_all_links(url):
  base_url = "https://www.visitrwanda.com/"
  driver.get(url)
  my_elements = driver.find_elements(By.TAG_NAME, 'a')
  all_links = []
  for element in my_elements:
      href = element.get_attribute("href")
      if href:
          # Check if the href starts with "http" or "https"
          if not href.startswith(("http://", "https://")):
              # If not, join it with the base_url
              href = urljoin(base_url, href)
          all_links.append(href)
  unique_links = list(set(all_links))
  return unique_links

In [ ]:
first_links_all = []
for link in nav_links:
  links = get_all_links(link)
  first_links_all += links


In [ ]:
unique_first_links = list(set(first_links_all))

In [ ]:
second_links_all = []
for index, link in enumerate(unique_first_links):
  try:
    print(f"scrap: {index} --- and link: {link}")
    links = get_all_links(link)
    second_links_all += links
  except:
    pass

In [ ]:
unique_second_links = list(set(second_links_all))

In [ ]:
len(unique_second_links)

942

In [ ]:
all_links = unique_second_links + unique_first_links + nav_links

In [ ]:
len(all_links)

1136

In [ ]:
final_links_all = []
for link in all_links:
        if link.startswith(("https://www.visitrwanda.com/", "http://www.visitrwanda.com/")):
            final_links_all.append(link)

In [ ]:
len(final_links_all)

604

In [ ]:
final_links_unique = list(set(final_links_all))

In [ ]:
len(final_links_unique)

423

In [ ]:
df = pd.DataFrame(final_links_unique)
df.head(3)

,0
0,http://www.visitrwanda.com/interests/ethnograp...
1,https://www.visitrwanda.com/destinations/kibeho/
2,http://www.visitrwanda.com/open-call-to-graffi...


In [ ]:
df.to_csv('visitrwandaall.csv', index=False)